Testing YOLO with CV2  
Created with example code from [here](https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html)

In [1]:
# YOLO object detection
import cv2 as cv
import numpy as np
import time

In [2]:
# img = cv.imread('Data/RBNR/set1_org/set1_78.JPG')

In [3]:
# Load names of classes and get random colors
classes = open('Data/SVHN/obj.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

In [4]:
# Give the configuration and weight files for the model and load the network.
configPath = 'Data/YOLO/num_reader/SVHN2-custom-yolov4-tiny-detector.cfg'
weightsPath = 'Data/YOLO/num_reader/SVHN2-custom-yolov4-tiny-detector_best.weights'

net = cv.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [5]:
cap = cv.VideoCapture(0)

In [ ]:
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # process frame   
    blob = cv.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # get detections
    net.setInput(blob)
    outputs = net.forward(ln)


    boxes = []
    confidences = []
    classIDs = []
    h, w = frame.shape[:2]

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.5:
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))
                classIDs.append(classID)

    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv.putText(frame, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    # Display the resulting frame
    cv.imshow('frame',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv.destroyAllWindows()